In [2]:
!pip install optuna
!wget https://github.com/oreilly-japan/ml-security-jp/raw/master/ch02/dataset.csv

     |████████████████████████████████| 308 kB 5.2 MB/s 
     |████████████████████████████████| 80 kB 9.0 MB/s 
     |████████████████████████████████| 209 kB 53.5 MB/s 
     |████████████████████████████████| 75 kB 4.8 MB/s 
     |████████████████████████████████| 49 kB 6.4 MB/s 
     |████████████████████████████████| 112 kB 54.8 MB/s 
     |████████████████████████████████| 149 kB 55.8 MB/s 
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11136 sha256=812fe59c31870fbffdbff08a8ffabb9ca4d1c934ae3ee63bb82f4559cda72692
  Stored in directory: /root/.cache/pip/wheels/9f/18/84/8f69f8b08169c7bae2dde6bd7daf0c19fca8c8e500ee620a28
Successfully built pyperclip
--2021-12-05 00:43:53--  https://github.com/oreilly-japan/ml-security-jp/raw/master/ch02/dataset.csv
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubuserconten

In [7]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import numpy as np
import optuna
from sklearn.model_selection import cross_validate

class Objective:
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __call__(self, trial):
        params = {
            'solver': trial.suggest_categorical('solver', ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']),
            'C': trial.suggest_loguniform('C', 0.0001, 10),
            'max_iter': trial.suggest_int('max_iter', 100, 100000)
        }
        model = LogisticRegression(**params)
        scores = cross_validate(model,
                                            X=self.X,
                                            y=self.y,
                                            scoring='accuracy',
                                            n_jobs=-1)
        return scores['test_score'].mean()

training_data = np.genfromtxt('dataset.csv', delimiter=',', dtype=np.int32)
X = training_data[:,:-1]
y = training_data[:, -1]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, shuffle=True, random_state=101
)

objective = Objective(X_train, y_train)
study = optuna.create_study(direction='maximize')
study.optimize(objective, timeout=60)
print('params:', study.best_params)

from sklearn.metrics import confusion_matrix, accuracy_score
model = LogisticRegression(
    solver = study.best_params['solver'],
    C = study.best_params['C'],
    max_iter = study.best_params['max_iter']
)
model.fit(X_train, y_train)
pred = model.predict(X_test)
print("Accuracy: {:.5f} %".format(100 * accuracy_score(y_test, pred)))
print(confusion_matrix(y_test, pred))

[I 2021-12-05 00:52:19,080] A new study created in memory with name: no-name-4bfe05dd-c8ef-4731-b300-43bfb4c33db3
[I 2021-12-05 00:52:19,489] Trial 0 finished with value: 0.9227730471237937 and parameters: {'solver': 'newton-cg', 'C': 0.0061441606942428595, 'max_iter': 27668}. Best is trial 0 with value: 0.9227730471237937.
[I 2021-12-05 00:52:19,914] Trial 1 finished with value: 0.9280877429025269 and parameters: {'solver': 'liblinear', 'C': 4.643225371316981, 'max_iter': 18639}. Best is trial 1 with value: 0.9280877429025269.
[I 2021-12-05 00:52:20,366] Trial 2 finished with value: 0.9282008650744726 and parameters: {'solver': 'lbfgs', 'C': 3.2766233080001776, 'max_iter': 62645}. Best is trial 2 with value: 0.9282008650744726.
[I 2021-12-05 00:52:20,635] Trial 3 finished with value: 0.9276352542147441 and parameters: {'solver': 'liblinear', 'C': 0.020386283935396727, 'max_iter': 4305}. Best is trial 2 with value: 0.9282008650744726.
[I 2021-12-05 00:52:20,861] Trial 4 finished with v

params: {'solver': 'saga', 'C': 0.0931240487312583, 'max_iter': 83112}
Accuracy: 92.35640 %
[[ 874   97]
 [  72 1168]]
